# SMU Logo bounding box detection
Design an algorithm to identify all images of SMU from a set of random images. A bounding box should be drawn around the SMU logo whenever it appears in an image.

When an SMU logo occurs in an image, a bounding box should be drawn around it. Logo identification will be evaluated by its F1 score.

### Setup
Install dependencies
```bash
pip install -r requirements.txt

```

In [1]:
%pip install -r requirements.txt

  Obtaining dependency information for ultralytics==8.0.196 from https://files.pythonhosted.org/packages/a8/97/d6d2592ba629ab41e18aaba14a4d75a5b56c40398a1c0b9e5979317798d0/ultralytics-8.0.196-py3-none-any.whl.metadata
  Obtaining dependency information for roboflow from https://files.pythonhosted.org/packages/32/98/117f2db9078fb16f9a7e1084ac904750f1bc877d5a85eff4f6c2d7f70a28/roboflow-1.1.21-py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python>=4.6.0 from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for torch>=1.8.0 from https://files.pythonhosted.org/packages/59/1f/4975d1ab3ed2244053876321ef65bc02935daed67da76c6e7d65900772a3/torch-2.2.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchvision>=0.9.0 from https://files.pythonhosted.org/packages/e7/45/419aa0b37254f1fd62b45bb63836066c

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.10.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Import packages

In [1]:
from ultralytics import YOLO
from roboflow import Roboflow
from PIL import Image
from dotenv import load_dotenv
import os

Load environment variables using dotenv
- `API_KEY` - API Key for Roboflow dataset
- `DATA_VERSION` - Version of dataset used
- `PATH_TO_DATA` - Absolute file path to the "SMU-Logo-Detection-1" folder. See .env.example for example

In [2]:
# Project settings
load_dotenv()
api_key = os.getenv('API_KEY')
data_version = os.getenv('DATA_VERSION')
path_to_data = f'{os.getenv("PATH_TO_PROJECT")}/tree-type-detection-{data_version}'

Load dataset into workspace

In [14]:
rf = Roboflow(api_key=api_key)
project = rf.workspace("tree-dataset-iftyz").project("tree-type-detection-9rfxy")
dataset = project.version(data_version).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to tree-type-detection-4 in yolov8:: 100%|██████████| 5776/5776 [00:15<00:00, 375.79it/s]


Update test, train and val values in data.yaml
This fixes a file not found bug

In [3]:
!python update_path.py

### [Training](https://docs.ultralytics.com/modes/train/#train-settings)
Use a trained model, saved in `/saved_models`, or train a model.
A pre-trained [model](https://github.com/ultralytics/ultralytics?tab=readme-ov-file) `yolov8n.pt` from Ultralytics should be used. Tune hyper-params such as learning rate and epochs.

If there is file not found error, check the `FILE_TO_PATH` env variable, and make sure that you've updated the `data.yml` to the absolute file path.

In [ ]:
# # Train model (Laptop)
# model = YOLO('yolov8n.pt') # pre-trained yolov8 nano
# results = model.train(data=f'{path_to_data}/data.yaml', epochs=10)

# # Train model (SMU GPU)
# model = YOLO('yolov8n.pt')
# results = model.train(data=f'{path_to_data}/data.yaml', epochs=10, batch=-1, device=0, workers=4)

In [4]:
# Use saved model
model = YOLO('./saved_models/v4_epoch13.pt')

After training, the model can be found in `/runs`. Save the `best.pt` model in the `/saved_models` folder.

### [Validation](https://docs.ultralytics.com/modes/val/#usage-examples)
Val mode is used for validating a YOLOv8 model after it has been trained. In this mode, the model is evaluated on a validation set to measure its accuracy and generalization performance. This mode can be used to tune the hyperparameters of the model to improve its performance.

In [5]:
model.val(data=f'{path_to_data}/data.yaml', conf=0.25, iou=0.6)
'''
1. Epochs: Vary from 7, 15 etc
2. Find a way to output the predicted labels into a folder. Compare the predicted labels and actual labels to check the metrics
3. Re-annotate if have time (those with very small logos)


4. annotate the rest of the images
5. after everything is annotated, augmentation and final training
'''

Ultralytics YOLOv8.0.196  Python-3.11.5 torch-2.2.1+cpu CPU (11th Gen Intel Core(TM) i7-11370H 3.30GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning C:\Users\WONG GINN MUNN\OneDrive\Desktop\WGM folder\SMU\Year 3 Semester 2\CS424\Project\Object Detection\CS424-Image-Perception\tree-type-detection-4\valid\labels... 576 images, 463 backgrounds, 0 corrupt: 100%|██████████| 576/576 [00:00<00:00, 592.43it/s]
val: New cache created: C:\Users\WONG GINN MUNN\OneDrive\Desktop\WGM folder\SMU\Year 3 Semester 2\CS424\Project\Object Detection\CS424-Image-Perception\tree-type-detection-4\valid\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [02:08<00:00,  3.58s/it]
                   all        576        129      0.939      0.721      0.846      0.674
Speed: 3.2ms preprocess, 205.3ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\val10


'\n1. Epochs: Vary from 7, 15 etc\n2. Find a way to output the predicted labels into a folder. Compare the predicted labels and actual labels to check the metrics\n3. Re-annotate if have time (those with very small logos)\n\n\n4. annotate the rest of the images\n5. after everything is annotated, augmentation and final training\n'

### [Test](https://docs.ultralytics.com/modes/predict/#why-use-ultralytics-yolo-for-inference)
Predict bounding boxes on test set

In [7]:
# Predict
prediction = model.predict(source=f"{path_to_data}/test/images", conf=0.25, iou=0.6, device='0')


image 1/190 D:\Disk D Documents\CS424\Project SMU Logo\SMU-Logo-Detection-3\test\images\IMG_9405_JPG.rf.3fb70211368fe25875d93bda4a2cd79b.jpg: 640x640 (no detections), 8.0ms
image 2/190 D:\Disk D Documents\CS424\Project SMU Logo\SMU-Logo-Detection-3\test\images\IMG_9416_JPG.rf.4660436543c8af105a3ddb473bc3dea3.jpg: 640x640 (no detections), 11.0ms
image 3/190 D:\Disk D Documents\CS424\Project SMU Logo\SMU-Logo-Detection-3\test\images\IMG_9431_JPG.rf.a152a6e86a53c1622a043d3cd90b3768.jpg: 640x640 (no detections), 9.0ms
image 4/190 D:\Disk D Documents\CS424\Project SMU Logo\SMU-Logo-Detection-3\test\images\IMG_9437_JPG.rf.221ff02915b847e53ea765481eeb565e.jpg: 640x640 (no detections), 11.0ms
image 5/190 D:\Disk D Documents\CS424\Project SMU Logo\SMU-Logo-Detection-3\test\images\IMG_9445_JPG.rf.e8ca272657213ae8b2714015d917da5a.jpg: 640x640 (no detections), 7.5ms
image 6/190 D:\Disk D Documents\CS424\Project SMU Logo\SMU-Logo-Detection-3\test\images\IMG_9450_JPG.rf.0f75bc032986300ea82077f3ab22

Save images to `/predictions` folder if bounding box exists.

In [9]:
images = {}
for i in range(0, len(prediction)):
    if prediction[i].boxes.shape[0] >= 1:
        im_rgb = prediction[i].plot(line_width=1)
        im_rgb = Image.fromarray(im_rgb[..., ::-1])  # RGB-order PIL image
        images[i] = im_rgb
        im_rgb.save(fp=f'{path_to_data}/../predictions/result{i}.jpg')
images

{21: <PIL.Image.Image image mode=RGB size=640x640>,
 38: <PIL.Image.Image image mode=RGB size=640x640>,
 40: <PIL.Image.Image image mode=RGB size=640x640>,
 45: <PIL.Image.Image image mode=RGB size=640x640>,
 46: <PIL.Image.Image image mode=RGB size=640x640>,
 47: <PIL.Image.Image image mode=RGB size=640x640>,
 49: <PIL.Image.Image image mode=RGB size=640x640>,
 50: <PIL.Image.Image image mode=RGB size=640x640>,
 51: <PIL.Image.Image image mode=RGB size=640x640>,
 61: <PIL.Image.Image image mode=RGB size=640x640>,
 62: <PIL.Image.Image image mode=RGB size=640x640>,
 63: <PIL.Image.Image image mode=RGB size=640x640>,
 66: <PIL.Image.Image image mode=RGB size=640x640>,
 67: <PIL.Image.Image image mode=RGB size=640x640>,
 68: <PIL.Image.Image image mode=RGB size=640x640>,
 83: <PIL.Image.Image image mode=RGB size=640x640>,
 85: <PIL.Image.Image image mode=RGB size=640x640>,
 109: <PIL.Image.Image image mode=RGB size=640x640>,
 110: <PIL.Image.Image image mode=RGB size=640x640>,
 128: <PIL